### This is from a Kaggle competition: https://www.kaggle.com/c/nlp-getting-started/submit

In [1]:
import pandas as pd

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [2]:
df_train = df_train

In [3]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
import sidetable
df_train.stb.missing(style=True)

,missing,total,percent
location,"2,533","7,613",33.27%
keyword,61,"7,613",0.80%
id,0,"7,613",0.00%
text,0,"7,613",0.00%
target,0,"7,613",0.00%


In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import numpy as np
def get_pad_sq(df):
    text = np.array(df.text)
    tokenizer = Tokenizer(num_words=len(df.text.unique()))
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    df = sequence.pad_sequences(sequences, maxlen=150)
    return df

In [6]:
X_train = get_pad_sq(df_train)
test = get_pad_sq(df_test)
y_train = np.array(df_train.target)

In [8]:
'''
Credit source: 
    https://www.tensorflow.org/tutorials/keras/keras_tuner
    https://medium.com/@mrunal68/text-sentiments-classification-with-cnn-and-lstm-f92652bc29fd
    https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html
'''

from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense, MaxPooling1D, LSTM, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

def build_model(hp):
    # create the model
    embedding_vector_length = 32
    vocab_size = 10000
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_vector_length, input_length=150))
    model.add(MaxPooling1D())
    model.add(LSTM(64, return_sequences = False))
    model.add(Dense(units=hp_units, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

In [9]:
import kerastuner as kt

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=16,
                     overwrite=True)

In [10]:
# checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy',verbose=0, save_best_only=True)
es = EarlyStopping(monitor='val_accuracy', verbose=0, patience=10)
callbacks_list = [es]

In [11]:
tuner.search(X_train, y_train, epochs=64, batch_size=32, validation_split=0.1, callbacks=callbacks_list)

Trial 30 Complete [00h 00m 39s]
val_accuracy: 0.9007092118263245

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 08m 03s
INFO:tensorflow:Oracle triggered exit


In [12]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=64, validation_split=0.2)
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
# Re-instantiate the hypermodel and train it with the optimal number of epochs from above.
hypermodel = tuner.hypermodel.build(best_hps)
# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch)
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

In [ ]:
# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch)

In [14]:
from keras.models import save_model
hypermodel.save('best_model.h5')

89/89 [==============================] - 4s 26ms/step - loss: 0.6886 - accuracy: 0.5839


In [16]:
# Load the model and predict
from keras.models import load_model
hypermodel = load_model('best_model.h5')
y_pred = hypermodel.predict(test)
y_pred

In [17]:
y_pred = pd.DataFrame([1 if x >= 0.5 else 0 for x in y_pred], columns=['target'], index=df_test.id)

In [18]:
# saving the dataframe 
y_pred.to_csv('Predictions.csv') 

In [19]:
import winsound
duration = 2000  # milliseconds
freq = 3000  # Hz
winsound.Beep(freq, duration)